DATA PRE-PROCESSING AND SPLITTING

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import boto3

# Load the dataset from S3
s3 = boto3.client('s3')
bucket_name = 'interactionsdataset'
file_key = 'synthetic_user_interactions.csv'

s3.download_file(bucket_name, file_key, '/tmp/synthetic_user_interactions.csv')
df = pd.read_csv('/tmp/synthetic_user_interactions.csv')

# Data preprocessing
df['timestamp'] = pd.to_datetime(df['timestamp'])
label_encoder_user = LabelEncoder()
label_encoder_attraction = LabelEncoder()
df['userId_encoded'] = label_encoder_user.fit_transform(df['userId'])
df['attractionId_encoded'] = label_encoder_attraction.fit_transform(df['attractionId'])

# One-hot encode interactionType
one_hot_encoder = OneHotEncoder(sparse_output=False)
interaction_type_encoded = one_hot_encoder.fit_transform(df[['interactionType']])
interaction_type_encoded_df = pd.DataFrame(interaction_type_encoded, columns=one_hot_encoder.get_feature_names_out(['interactionType']))
df = pd.concat([df, interaction_type_encoded_df], axis=1)

# Feature engineering
df['user_interaction_count'] = df.groupby('userId_encoded')['userId_encoded'].transform('count')
df['attraction_popularity'] = df.groupby('attractionId_encoded')['attractionId_encoded'].transform('count')
max_timestamp = df['timestamp'].max()
df['interaction_recency'] = (max_timestamp - df['timestamp']).dt.days
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour_of_day'] = df['timestamp'].dt.hour

# Select features for the model
features = ['userId_encoded', 'attractionId_encoded', 'user_interaction_count', 
            'attraction_popularity', 'interaction_recency', 'day_of_week', 'hour_of_day']

# Target variable
target = 'interactionType_view'

# Verify the columns
print(df.columns)

# Prepare training and testing data
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


Index(['userId', 'attractionId', 'interactionType', 'timestamp',
       'userId_encoded', 'attractionId_encoded', 'interactionType_view',
       'user_interaction_count', 'attraction_popularity',
       'interaction_recency', 'day_of_week', 'hour_of_day'],
      dtype='object')


((800, 7), (200, 7), (800,), (200,))

UPLOADING PRE-PROCESSED DATA TO THE S3 BUCKET

In [4]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'interactionsdataset'
prefix = 'sagemaker/collaborative-filtering'

# Upload training data to S3
train_file = 'train_data.csv'
test_file = 'test_data.csv'
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)
df_train.to_csv(f'/tmp/{train_file}', index=False)
df_test.to_csv(f'/tmp/{test_file}', index=False)
s3.upload_file(f'/tmp/{train_file}', bucket_name, f'{prefix}/{train_file}')
s3.upload_file(f'/tmp/{test_file}', bucket_name, f'{prefix}/{test_file}')


PREPARING TESTING AND TRAINING DATA

In [8]:
import sagemaker.amazon.common as smac
import boto3
import numpy as np
import io

# Prepare the training data
train_features = X_train.to_numpy().astype('float32')
train_labels = y_train.to_numpy().astype('float32').flatten()  # Ensure labels are a 1D array

# Convert to RecordIO Protobuf format
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels)
buf.seek(0)

# Upload the training data to S3
bucket = 'interactionsdataset'
prefix = 'sagemaker/collaborative-filtering'
key = 'train_data.protobuf'
s3_object = f'{prefix}/{key}'
boto3.resource('s3').Bucket(bucket).Object(s3_object).upload_fileobj(buf)

# Prepare the testing data
test_features = X_test.to_numpy().astype('float32')
test_labels = y_test.to_numpy().astype('float32').flatten()  # Ensure labels are a 1D array

# Convert to RecordIO Protobuf format
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features, test_labels)
buf.seek(0)

# Upload the testing data to S3
test_key = 'test_data.protobuf'
test_s3_object = f'{prefix}/{test_key}'
boto3.resource('s3').Bucket(bucket).Object(test_s3_object).upload_fileobj(buf)


TRAINING THE MODEL

In [9]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

# Define the role and S3 paths
role = get_execution_role()
output_path = f's3://{bucket}/{prefix}/output'

# Create and configure the Factorization Machines estimator
fm = Estimator(
    image_uri=sagemaker.image_uris.retrieve("factorization-machines", boto3.Session().region_name),
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=output_path
)

fm.set_hyperparameters(
    feature_dim=train_features.shape[1],
    num_factors=10,
    predictor_type='binary_classifier',
    mini_batch_size=100
)

# Train the model
fm.fit({'train': f's3://{bucket}/{s3_object}', 'test': f's3://{bucket}/{test_s3_object}'})


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: factorization-machines-2024-07-28-12-38-13-255


2024-07-28 12:38:13 Starting - Starting the training job...
2024-07-28 12:38:28 Starting - Preparing the instances for training...
2024-07-28 12:38:55 Downloading - Downloading input data...
2024-07-28 12:39:25 Downloading - Downloading the training image.........
2024-07-28 12:41:07 Training - Training image download completed. Training in progress....
2024-07-28 12:41:37 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[07/28/2024 12:41:30 INFO 140663685162816] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'epochs': 1, 'mini_batch_size': '1000', 'use_bias': 'true', 'use_linear': 'true', 'bias_lr': '0.1', 'linear_lr': '0.001', 'factors_lr': '0.0001', 'bias_wd': '0.


2024-07-28 12:41:50 Completed - Training job completed
Training seconds: 175
Billable seconds: 175


DEPLOYING THE MODEL

In [16]:
#Deploying the model
predictor = fm.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: factorization-machines-2024-07-28-12-45-44-346
INFO:sagemaker:Creating endpoint-config with name factorization-machines-2024-07-28-12-45-44-346
INFO:sagemaker:Creating endpoint with name factorization-machines-2024-07-28-12-45-44-346


------------!

PREPARING TESTING DATA

In [18]:
import boto3
import pandas as pd
from io import StringIO
from sagemaker.predictor import Predictor
from sklearn.metrics import mean_squared_error

# Assuming test_features is a numpy array and we need to convert it to CSV format
test_features_df = pd.DataFrame(test_features)

# Convert the DataFrame to CSV
csv_buffer = StringIO()
test_features_df.to_csv(csv_buffer, header=False, index=False)

# Get the CSV data as a string
csv_data = csv_buffer.getvalue()


TESTING THE MODEL

In [20]:
import boto3
import pandas as pd
from io import StringIO
import sagemaker.amazon.common as smac
import numpy as np

# Prepare a single test example
single_test_feature = X_test.iloc[0].to_numpy().astype('float32').reshape(1, -1)
single_test_label = y_test.iloc[0].astype('float32').reshape(1, -1)

# Convert the single test example to RecordIO Protobuf format
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, single_test_feature)
buf.seek(0)
payload = buf.getvalue()

# Create a SageMaker runtime client
runtime_client = boto3.client('runtime.sagemaker')

# Define the endpoint name
endpoint_name = 'factorization-machines-2024-07-28-12-45-44-346'

# Invoke the endpoint with the correct content type
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-recordio-protobuf',
    Body=payload
)

# Parse the response
result = response['Body'].read().decode('utf-8')
print('Prediction result:', result)


Prediction result: {"predictions": [{"score": 1.0, "predicted_label": 1.0}]}


MODEL EVALUATION

In [21]:
import boto3
import pandas as pd
from io import BytesIO
import sagemaker.amazon.common as smac
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize boto3 client
s3 = boto3.client('s3')

# S3 bucket and file details
bucket_name = 'interactionsdataset'
file_key = 'synthetic_user_interactions.csv'

# Download the file from S3
s3_object = s3.get_object(Bucket=bucket_name, Key=file_key)
s3_data = s3_object['Body'].read().decode('utf-8')

# Load the data into a pandas DataFrame
df = pd.read_csv(StringIO(s3_data))

# Data preprocessing
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['userId_encoded'] = df['userId'].astype('category').cat.codes
df['attractionId_encoded'] = df['attractionId'].astype('category').cat.codes
df['interactionType_view'] = df['interactionType'].apply(lambda x: 1 if x == 'view' else 0)

# Feature engineering
df['user_interaction_count'] = df.groupby('userId_encoded')['userId_encoded'].transform('count')
df['attraction_popularity'] = df.groupby('attractionId_encoded')['attractionId_encoded'].transform('count')
max_timestamp = df['timestamp'].max()
df['interaction_recency'] = (max_timestamp - df['timestamp']).dt.days
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour_of_day'] = df['timestamp'].dt.hour

# Select features for the model
features = ['userId_encoded', 'attractionId_encoded', 'user_interaction_count', 
            'attraction_popularity', 'interaction_recency', 'day_of_week', 'hour_of_day']
target = 'interactionType_view'

# Prepare training and testing data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prepare the test data in RecordIO Protobuf format
test_features = X_test.to_numpy().astype('float32')
test_labels = y_test.to_numpy().astype('float32').flatten()
buf = BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features)
buf.seek(0)
payload = buf.getvalue()

# Create a SageMaker runtime client
runtime_client = boto3.client('runtime.sagemaker')

# Define the endpoint name
endpoint_name = 'factorization-machines-2024-07-28-12-45-44-346'

# Invoke the endpoint with the correct content type
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-recordio-protobuf',
    Body=payload
)

# Parse the response
result = response['Body'].read().decode('utf-8')

# Extract predictions from the JSON response
import json
predictions = json.loads(result)
predicted_labels = [pred['predicted_label'] for pred in predictions['predictions']]

# Calculate RMSE
rmse = mean_squared_error(test_labels, predicted_labels, squared=False)


Test RMSE: 0.0


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
import boto3
import pandas as pd
from io import BytesIO
import sagemaker.amazon.common as smac
from sklearn.metrics import mean_squared_error
import numpy as np
import json

# Initialize boto3 client
s3 = boto3.client('s3')

# S3 bucket and file details
bucket_name = 'interactionsdataset'
file_key = 'synthetic_user_interactions.csv'

# Download the file from S3
s3_object = s3.get_object(Bucket=bucket_name, Key=file_key)
s3_data = s3_object['Body'].read().decode('utf-8')

# Load the data into a pandas DataFrame
df = pd.read_csv(StringIO(s3_data))

# Data preprocessing
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['userId_encoded'] = df['userId'].astype('category').cat.codes
df['attractionId_encoded'] = df['attractionId'].astype('category').cat.codes
df['interactionType_view'] = df['interactionType'].apply(lambda x: 1 if x == 'view' else 0)

# Feature engineering
df['user_interaction_count'] = df.groupby('userId_encoded')['userId_encoded'].transform('count')
df['attraction_popularity'] = df.groupby('attractionId_encoded')['attractionId_encoded'].transform('count')
max_timestamp = df['timestamp'].max()
df['interaction_recency'] = (max_timestamp - df['timestamp']).dt.days
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour_of_day'] = df['timestamp'].dt.hour

# Select features for the model
features = ['userId_encoded', 'attractionId_encoded', 'user_interaction_count', 
            'attraction_popularity', 'interaction_recency', 'day_of_week', 'hour_of_day']
target = 'interactionType_view'

# Prepare training and testing data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prepare the test data in RecordIO Protobuf format
test_features = X_test.to_numpy().astype('float32')
test_labels = y_test.to_numpy().astype('float32').flatten()
buf = BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features)
buf.seek(0)
payload = buf.getvalue()

# Create a SageMaker runtime client
runtime_client = boto3.client('runtime.sagemaker')

# Define the endpoint name
endpoint_name = 'factorization-machines-2024-07-28-12-45-44-346'

# Invoke the endpoint with the correct content type
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-recordio-protobuf',
    Body=payload
)

# Parse the response
result = response['Body'].read().decode('utf-8')
predictions = json.loads(result)
predicted_labels = [pred['predicted_label'] for pred in predictions['predictions']]

# Print the first 10 test labels and predicted labels for comparison
print("Test Labels (first 10):", test_labels[:10])
print("Predicted Labels (first 10):", predicted_labels[:10])

# Calculate RMSE
rmse = mean_squared_error(test_labels, predicted_labels, squared=False)
print('Test RMSE:', rmse)

# Check the distribution of the target variable in the test set
print("Distribution of test labels:", np.bincount(test_labels.astype(int)))
print("Distribution of predicted labels:", np.bincount(np.array(predicted_labels).astype(int)))


Test Labels (first 10): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Predicted Labels (first 10): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Test RMSE: 0.0
Distribution of test labels: [  0 200]
Distribution of predicted labels: [  0 200]


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
# Check the overall distribution of the target variable in the entire dataset
print("Overall distribution of interactionType_view in the dataset:")
print(df['interactionType_view'].value_counts())


Overall distribution of interactionType_view in the dataset:
interactionType_view
1    1000
Name: count, dtype: int64


GETTING RECOMMENDATIONS

In [3]:
import numpy as np
import boto3
from sagemaker.predictor import Predictor
import sagemaker.amazon.common as smac
from io import BytesIO
import pandas as pd
import json

# Initialize boto3 client
s3 = boto3.client('s3')

# S3 bucket and file details
bucket_name = 'interactionsdataset'
file_key = 'synthetic_user_interactions.csv'

# Download the file from S3
s3_object = s3.get_object(Bucket=bucket_name, Key=file_key)
s3_data = s3_object['Body'].read().decode('utf-8')

# Load the data into a pandas DataFrame
df = pd.read_csv(StringIO(s3_data))

# Data preprocessing
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['userId_encoded'] = df['userId'].astype('category').cat.codes
df['attractionId_encoded'] = df['attractionId'].astype('category').cat.codes
df['interactionType_view'] = df['interactionType'].apply(lambda x: 1 if x == 'view' else 0)

# Feature engineering
df['user_interaction_count'] = df.groupby('userId_encoded')['userId_encoded'].transform('count')
df['attraction_popularity'] = df.groupby('attractionId_encoded')['attractionId_encoded'].transform('count')
max_timestamp = df['timestamp'].max()
df['interaction_recency'] = (max_timestamp - df['timestamp']).dt.days
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour_of_day'] = df['timestamp'].dt.hour

# Select features for the model
features = ['userId_encoded', 'attractionId_encoded', 'user_interaction_count', 
            'attraction_popularity', 'interaction_recency', 'day_of_week', 'hour_of_day']

# Define a function to get recommendations
def get_recommendations(user_id, n_recommendations=5):
    # Encode the user_id
    user_encoded = df.loc[df['userId'] == user_id, 'userId_encoded'].values[0]

    # Prepare the data for all attractions for this user
    user_data = []
    for attraction_id in df['attractionId_encoded'].unique():
        interaction_data = [
            user_encoded,
            attraction_id,
            df[df['attractionId_encoded'] == attraction_id]['user_interaction_count'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['attraction_popularity'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['interaction_recency'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['day_of_week'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['hour_of_day'].values[0]
        ]
        user_data.append(interaction_data)

    # Convert to numpy array and prepare payload
    user_data_np = np.array(user_data).astype('float32')
    buf = BytesIO()
    smac.write_numpy_to_dense_tensor(buf, user_data_np)
    buf.seek(0)
    payload = buf.getvalue()

    # Create a SageMaker runtime client
    runtime_client = boto3.client('runtime.sagemaker')

    # Define the endpoint name
    endpoint_name = 'factorization-machines-2024-07-28-12-45-44-346'

    # Invoke the endpoint with the correct content type
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/x-recordio-protobuf',
        Body=payload
    )

    # Parse the response
    result = response['Body'].read().decode('utf-8')
    predictions = json.loads(result)
    scores = [pred['score'] for pred in predictions['predictions']]

    # Get the top n recommendations
    top_indices = np.argsort(scores)[-n_recommendations:][::-1]
    top_attractions = df.loc[df['attractionId_encoded'].isin(top_indices), 'attractionId'].unique()

    return user_id, top_attractions

# Example usage
user_id = df['userId'].iloc[0]  # Replace with the actual user ID you want recommendations for
user_id, recommendations = get_recommendations(user_id)
print(f'Recommendations for user {user_id}:', recommendations)


Recommendations for user 4b4d5711-5539-44ef-a4fc-5c55f5d8be37: ['Volcanoes NP (Gorilla trekking)' 'Amahoro Stadium' 'BK Arena'
 'Underground caves' 'Amashyuza hot springs']


In [4]:
import numpy as np
import boto3
from sagemaker.predictor import Predictor
import sagemaker.amazon.common as smac
from io import BytesIO, StringIO
import pandas as pd
import json

# Initialize boto3 client
s3 = boto3.client('s3')

# S3 bucket and file details
bucket_name = 'interactionsdataset'
file_key = 'synthetic_user_interactions.csv'

# Download the file from S3
s3_object = s3.get_object(Bucket=bucket_name, Key=file_key)
s3_data = s3_object['Body'].read().decode('utf-8')

# Load the data into a pandas DataFrame
df = pd.read_csv(StringIO(s3_data))

# Data preprocessing
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['userId_encoded'] = df['userId'].astype('category').cat.codes
df['attractionId_encoded'] = df['attractionId'].astype('category').cat.codes
df['interactionType_view'] = df['interactionType'].apply(lambda x: 1 if x == 'view' else 0)

# Feature engineering
df['user_interaction_count'] = df.groupby('userId_encoded')['userId_encoded'].transform('count')
df['attraction_popularity'] = df.groupby('attractionId_encoded')['attractionId_encoded'].transform('count')
max_timestamp = df['timestamp'].max()
df['interaction_recency'] = (max_timestamp - df['timestamp']).dt.days
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour_of_day'] = df['timestamp'].dt.hour

# Select features for the model
features = ['userId_encoded', 'attractionId_encoded', 'user_interaction_count', 
            'attraction_popularity', 'interaction_recency', 'day_of_week', 'hour_of_day']

# Define a function to get recommendations
def get_recommendations(user_id, n_recommendations=5):
    # Encode the user_id
    user_encoded = df.loc[df['userId'] == user_id, 'userId_encoded'].values[0]

    # Prepare the data for all attractions for this user
    user_data = []
    for attraction_id in df['attractionId_encoded'].unique():
        interaction_data = [
            user_encoded,
            attraction_id,
            df[df['attractionId_encoded'] == attraction_id]['user_interaction_count'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['attraction_popularity'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['interaction_recency'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['day_of_week'].values[0],
            df[df['attractionId_encoded'] == attraction_id]['hour_of_day'].values[0]
        ]
        user_data.append(interaction_data)

    # Convert to numpy array and prepare payload
    user_data_np = np.array(user_data).astype('float32')
    buf = BytesIO()
    smac.write_numpy_to_dense_tensor(buf, user_data_np)
    buf.seek(0)
    payload = buf.getvalue()

    # Create a SageMaker runtime client
    runtime_client = boto3.client('runtime.sagemaker')

    # Define the endpoint name
    endpoint_name = 'factorization-machines-2024-07-28-12-45-44-346'

    # Invoke the endpoint with the correct content type
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/x-recordio-protobuf',
        Body=payload
    )

    # Parse the response
    result = response['Body'].read().decode('utf-8')
    predictions = json.loads(result)
    scores = [pred['score'] for pred in predictions['predictions']]

    # Get the top n recommendations
    top_indices = np.argsort(scores)[-n_recommendations:][::-1]
    top_attractions = df['attractionId_encoded'].unique()[top_indices]
    top_scores = [scores[i] for i in top_indices]

    # Map encoded attraction IDs back to original attraction IDs
    top_attractions_ids = df.loc[df['attractionId_encoded'].isin(top_attractions), 'attractionId'].unique()
    
    # Return user_id, top_attractions_ids and top_scores
    return user_id, list(zip(top_attractions_ids, top_scores))

# Example usage
user_id = df['userId'].iloc[0]  # Replace with the actual user ID you want recommendations for
user_id, recommendations = get_recommendations(user_id)
print(f'Recommendations for user {user_id}:')
for attraction, score in recommendations:
    print(f'Attraction ID: {attraction}')


Recommendations for user 4b4d5711-5539-44ef-a4fc-5c55f5d8be37:
Attraction ID: Nyandungu park
Attraction ID: Kigali Memorial Site
Attraction ID: Cathedral
Attraction ID: Royal balloon
Attraction ID: Amashyuza hot springs


CHECKING EXISTING ENPOINTS

In [ ]:
import boto3

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List endpoints
response = sagemaker_client.list_endpoints()

# Print the endpoint names
for endpoint in response['Endpoints']:
    print(endpoint['EndpointName'])


DELETING ENDPOINTS

In [ ]:
import boto3

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Specify the endpoint name you want to delete
endpoint_name = 'factorization-machines-2024-07-25-19-23-18-370'

# Delete the endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

print(f"Endpoint '{endpoint_name}' has been deleted.")
